# Asana to Scoro Migration

This notebook runs the complete migration process from Asana to Scoro.

## Setup Instructions

1. Ensure you have a `.env` file in the project root with:
   - `ASANA_ACCESS_TOKEN=your_token`
   - `SCORO_API_KEY=your_key`
   - `SCORO_COMPANY_NAME=your_company`

2. Install dependencies:
   ```bash
   pip install -r requirements.txt
   ```

3. Run the cells below in order.


In [1]:
# Import required libraries
import sys
import os
import json
from datetime import datetime

# Add project root to path
sys.path.insert(0, os.path.abspath('.'))

# Import project modules
from clients import AsanaClient, ScoroClient
from models import MigrationSummary
from exporters import export_asana_project
from transformers import transform_data, reset_task_tracker, get_deduplication_stats
from importers import import_to_scoro
from utils import logger

print("✓ All imports successful")


✓ All imports successful


## Configuration

Configure the project you want to migrate. You can use either:
- **Project GID** (recommended, faster): Direct project identifier
- **Project Name**: Search by name (slower)

Set the values below:


In [2]:
# Configuration - Modify these values as needed
PROJECT_GID = "1207816263671761"  # Set to None if using project_name instead
WORKSPACE_GID = "10447183158961"
PROJECT_NAME = None  # Set to project name if not using GID

# Display configuration
print("Migration Configuration:")
print(f"  Project GID: {PROJECT_GID}")
print(f"  Workspace GID: {WORKSPACE_GID}")
print(f"  Project Name: {PROJECT_NAME}")
print(f"  Using: {'Project GID' if PROJECT_GID else 'Project Name'}")


Migration Configuration:
  Project GID: 1207816263671761
  Workspace GID: 10447183158961
  Project Name: None
  Using: Project GID


## Phase 1: Initialize Clients and Test Connections


In [ ]:
# Initialize clients
print("="*60)
print("Initializing API clients...")
print("="*60)

try:
    # Initialize Asana client
    asana_client = AsanaClient()
    print("✓ Asana client initialized")
    
    # Test Asana connection
    print("\nTesting Asana API connection...")
    if asana_client.test_connection():
        print("✓ Asana connection test successful")
    else:
        print("✗ Asana connection test failed")
        print("\nPossible issues:")
        print("  1. Your ASANA_ACCESS_TOKEN may be invalid or expired")
        print("  2. The token may not have the required permissions")
        print("  3. Check that your .env file is in the correct location")
        raise ValueError("Asana connection failed")
        
except ValueError as e:
    print(f"✗ Authentication Error: {e}")
    print("\nPlease ensure your .env file contains a valid ASANA_ACCESS_TOKEN.")
    print("You can create a Personal Access Token at: https://app.asana.com/0/developer-console")
    raise

# Initialize Scoro client
scoro_client = ScoroClient()

# Test Scoro connection
print("\nTesting Scoro connection...")
try:
    scoro_projects = scoro_client.list_projects()
    print(f"✓ Connected to Scoro. Found {len(scoro_projects)} existing projects.")
    if scoro_projects:
        print("  Sample projects:")
        for proj in scoro_projects[:5]:  # Show first 5
            print(f"    - {proj.get('name', 'Unknown')}")
except Exception as e:
    print(f"✗ Error connecting to Scoro: {e}")
    raise

print("\n✓ All clients initialized and tested successfully")


## Phase 2: Export from Asana


In [ ]:
# Export project from Asana
print("="*60)
print("PHASE 1: EXPORT FROM ASANA")
print("="*60)

if PROJECT_GID:
    print(f"\nExporting project from Asana using GID: {PROJECT_GID}...")
    asana_data = export_asana_project(asana_client, project_gid=PROJECT_GID, workspace_gid=WORKSPACE_GID)
elif PROJECT_NAME:
    print(f"\nExporting project from Asana: '{PROJECT_NAME}'...")
    asana_data = export_asana_project(asana_client, project_name=PROJECT_NAME, workspace_gid=WORKSPACE_GID)
else:
    raise ValueError("Either PROJECT_GID or PROJECT_NAME must be provided")

if not asana_data:
    error_msg = f"✗ Project (GID: {PROJECT_GID if PROJECT_GID else 'name: ' + PROJECT_NAME}) not found or could not be exported."
    print(error_msg)
    raise ValueError(error_msg)

print(f"✓ Successfully exported project with {len(asana_data.get('tasks', []))} tasks")

# Display project details
if asana_data.get('project'):
    proj = asana_data['project']
    print("\nProject Details Retrieved:")
    print(f"  Name: {proj.get('name', 'N/A')}")
    print(f"  GID: {proj.get('gid', 'N/A')}")
    print(f"  Created: {proj.get('created_at', 'N/A')}")
    print(f"  Modified: {proj.get('modified_at', 'N/A')}")

# Display task count
tasks = asana_data.get('tasks', [])
print(f"\nTotal tasks exported: {len(tasks)}")


## Phase 3: Transform Data


In [ ]:
# Initialize migration summary and reset task tracker
summary = MigrationSummary()
reset_task_tracker()
print("Task deduplication tracker reset")

# Transform data
print("\n" + "="*60)
print("PHASE 2: TRANSFORM DATA")
print("="*60)
print("\nTransforming data...")

transformed_data = transform_data(asana_data, summary)

print("✓ Data transformation completed")
print(f"\nTransformation Summary:")
print(f"  Total items processed: {summary.total_items}")
print(f"  Succeeded: {summary.succeeded}")
print(f"  Failed: {summary.failed}")

if transformed_data:
    print(f"\nTransformed Data Structure:")
    print(f"  Projects: {len(transformed_data.get('projects', []))}")
    print(f"  Tasks: {len(transformed_data.get('tasks', []))}")


## Phase 4: Import to Scoro

**Note**: This will create actual projects and tasks in Scoro. Make sure you're ready to proceed.


In [ ]:
# Import to Scoro
print("="*60)
print("PHASE 3: IMPORT TO SCORO")
print("="*60)
print("\n" + "-"*60)
print("NOTE: Import to Scoro is currently enabled.")
print("-"*60)

print("\nImporting to Scoro...")
import_results = import_to_scoro(scoro_client, transformed_data, summary)
print("✓ Import completed")


## Phase 5: Results and Summary


In [ ]:
# Print migration summary
print("="*60)
print("MIGRATION SUMMARY")
print("="*60)
summary.print_summary()

# Print deduplication statistics
dedup_stats = get_deduplication_stats()
if dedup_stats['total_tasks_seen'] > 0:
    print("="*60)
    print("DEDUPLICATION STATISTICS")
    print("="*60)
    print(f"Total unique tasks seen: {dedup_stats['total_tasks_seen']}")
    print(f"  - From client projects: {dedup_stats['client_project_tasks']}")
    print(f"  - From team member projects: {dedup_stats['team_member_project_tasks']}")
    print(f"Deduplication: {dedup_stats['total_tasks_seen']} unique tasks "
          f"({dedup_stats['client_project_tasks']} client, "
          f"{dedup_stats['team_member_project_tasks']} team member)")
    print("="*60)


## Save Export Data

The exported data is saved to a JSON file for inspection.


In [ ]:
# Save export data to file for inspection
print("Saving exported data to file...")
output_file = f"asana_export_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(asana_data, f, indent=2, default=str)
print(f"✓ Exported data saved to: {output_file}")

print("\n" + "="*60)
print("Migration process completed")
print("="*60)


## Optional: View Exported Data

You can inspect the exported data structure below:


In [ ]:
# Display a sample of the exported data
import json

print("Sample of exported Asana data:")
print(json.dumps({
    'project': asana_data.get('project', {}),
    'task_count': len(asana_data.get('tasks', [])),
    'sample_task': asana_data.get('tasks', [{}])[0] if asana_data.get('tasks') else None
}, indent=2, default=str))


## Optional: View Transformed Data

You can inspect the transformed data structure below:


In [ ]:
# Display a sample of the transformed data
if transformed_data:
    print("Sample of transformed data:")
    print(json.dumps({
        'projects_count': len(transformed_data.get('projects', [])),
        'tasks_count': len(transformed_data.get('tasks', [])),
        'sample_project': transformed_data.get('projects', [{}])[0] if transformed_data.get('projects') else None,
        'sample_task': transformed_data.get('tasks', [{}])[0] if transformed_data.get('tasks') else None
    }, indent=2, default=str))
